In [1]:
import torch
import torch.nn as nn

import numpy as np

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
# 0. prepare data
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

In [3]:
print(X.shape)

(569, 30)


In [4]:
n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.2, random_state=1234)

# scale out features
sc = StandardScaler() # zero mean, variance
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1) # column vector
y_test = y_test.view(y_test.shape[0], 1)

In [5]:
# 1. define model
# f = w*x + b

class LogisticRegression(nn.Module):
    
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
        
    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted
    
model = LogisticRegression(n_features)

In [6]:
# 2. define loss and optimizer
learning_rate = 0.01
criterion = nn.BCELoss()  # binary cross entropy
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [7]:
# 3. training loop

num_epochs = 100

for epoch in range(num_epochs):
    # forward pass and loss
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)
    
    # backward pass
    loss.backward()
    
    optimizer.step()
    
    optimizer.zero_grad() # empty our gradient (flush .grad attribute)
    
    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

epoch: 10, loss = 0.4575
epoch: 20, loss = 0.3950
epoch: 30, loss = 0.3522
epoch: 40, loss = 0.3207
epoch: 50, loss = 0.2964
epoch: 60, loss = 0.2769
epoch: 70, loss = 0.2610
epoch: 80, loss = 0.2475
epoch: 90, loss = 0.2360
epoch: 100, loss = 0.2261


In [8]:
# 4. evaluation our model

with torch.no_grad(): # this is not part of computation graph
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

accuracy = 0.9035
